In [1]:
%load_ext autoreload
%autoreload

## This expands a notebook to full width
from IPython.display import display, HTML

display(HTML("<style>.container { width:90% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))
## Show Python Version
import sys
print("Python: {0}".format(sys.version))

## Show Current Time
import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.10.9 | packaged by conda-forge | (main, Jan 11 2023, 15:15:40) [MSC v.1916 64 bit (AMD64)]
Notebook Last Run Initiated: 2023-03-13 19:40:21.810241


In [2]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import random
import tensorflow as tf

import random

from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.utils import Sequence

from tensorflow import keras

from sklearn.metrics.pairwise import cosine_similarity

In [3]:
MAX_SEQUENCE_LENGTH = 14

reserved_tokens = ["[PAD]", "[UNK]", "[START]", "[END]"]

seed = 23

learning_rate = 0.001
weight_decay = 0.0001
batch_size = 128
num_epochs = 500
projection_dim = 300
num_heads = 10

transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers

transformer_layers = 18

embed_dim = projection_dim 
embeddings_shape = (1,embed_dim)

mlp_head_units = [2048, 1024]  # Size of the dense layers of the final classifier

ckp_path = 'models/Model_Embedding_transformers_v3.hdf5'



In [4]:
tf.keras.utils.set_random_seed(seed)
np.random.seed(seed)

In [5]:
class DataGenerator(Sequence):
        
    def __init__(self, df_X, batch_size=32, shuffle=False):
        self.batch_size = batch_size
        self.df_X = df_X
        self.indices = self.df_X.index.tolist()
        self.vocab_size = vocab_size
        self.shuffle = shuffle
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.floor(len(self.indices) / self.batch_size))

    def __getitem__(self, index):
        index = self.index[index * self.batch_size:(index + 1) * self.batch_size]
        batch = [self.indices[k] for k in index]
        
        X, y = self.__get_data(batch)
        return X, y
    
    def n(self):
        return len(self.indices)
    
    def on_epoch_end(self):
        self.index = np.arange(len(self.indices))
        if self.shuffle == True:
            np.random.shuffle(self.index)

    def __get_data(self, batch):
        X1 = []
        y1 = []
        
        for i, id in enumerate(batch):
            
            column = random.randrange(MAX_SEQUENCE_LENGTH)
            
            # Data
            texts = self.df_X.iloc[self.indices[id]]
            
            #Labels
            label = texts[column]
            texts[column] = 0

            X1.append(texts)
            y1.append(label)

                            
        return np.array(X1), np.array(y1).reshape(self.batch_size,1)

In [6]:
# Input vector, returns nearest word(s)
def Cosine_Similarity(word,weight,word_to_index,vocab_size,index_to_word):
    
    #Get the index of the word from the dictionary
    index = word_to_index[word]
    
    #Get the correspondin weights for the word
    word_vector_1 = weight[index]
    
    word_similarity = {}

    for i in range(vocab_size):
        
        j = i
        
        word_vector_2 = weight[j]
        
        theta_sum = np.dot(word_vector_1, word_vector_2)
        theta_den = np.linalg.norm(word_vector_1) * np.linalg.norm(word_vector_2)
        theta = theta_sum / theta_den
        
        word = index_to_word[j]
        word_similarity[word] = theta
    
    return word_similarity #words_sorted

In [7]:
class PositionEmbedding(layers.Layer):
    def __init__(self, maxlen, embed_dim):
        super().__init__()
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        return positions + x

In [8]:
class PositionEmbeddingFixedWeights(layers.Layer):
    def __init__(self, sequence_length, output_dim, **kwargs):
        super(PositionEmbeddingFixedWeights, self).__init__(**kwargs)
        
        position_embedding_matrix = self.get_position_encoding(sequence_length, output_dim) 
        
        self.position_embedding_layer = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim,
            weights=[position_embedding_matrix],
            trainable=False
        )
             
    def get_position_encoding(self, seq_len, d, n=10000):
        P = np.zeros((seq_len, d))
        for k in range(seq_len):
            for i in np.arange(int(d/2)):
                denominator = np.power(n, 2*i/d)
                P[k, 2*i] = np.sin(k/denominator)
                P[k, 2*i+1] = np.cos(k/denominator)
        return P


    def call(self, inputs):        
        position_indices = tf.range(tf.shape(inputs)[1])
        embedded_indices = self.position_embedding_layer(position_indices)
        return inputs + embedded_indices

In [9]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [10]:
def create_classifier(learning_rate):
    
    input_text  = layers.Input(shape=(MAX_SEQUENCE_LENGTH-1,),dtype="int32",name='input_text')
    
    embeddings = tf.keras.layers.Embedding(input_dim=vocab_size, input_length=MAX_SEQUENCE_LENGTH, output_dim=embed_dim,name='embeddings')(input_text)
    
    embeddings = PositionEmbeddingFixedWeights(sequence_length=MAX_SEQUENCE_LENGTH-1,output_dim=embed_dim)(embeddings)
    
    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(embeddings)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1)(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, embeddings])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        # Skip connection 2.
        embeddings = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = layers.LayerNormalization(epsilon=1e-6)(embeddings)
    representation = layers.GlobalAveragePooling1D()(representation)
    representation = layers.Dropout(0.1)(representation)
    
    # Add MLP.
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.1)
    
    # Classify outputs.
    outputs = layers.Dense(num_classes,activation='softmax', name='activation')(features)
    
    # Create the Keras model.
    model = Model(inputs=input_text, outputs=outputs,name='Customer_Code_Embeddings')
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = learning_rate),loss="sparse_categorical_crossentropy")
    
    return model

In [11]:
# Input vector, returns nearest word(s)
def Cosine_Similarity(word,weight,word_to_index,vocab_size,index_to_word):
    
    #Get the index of the word from the dictionary
    index = word_to_index[word]
    
    #Get the correspondin weights for the word
    word_vector_1 = weight[index]
    
    word_similarity = {}

    for i in range(vocab_size):
        
        j = i
        
        word_vector_2 = weight[j]
        
        theta_sum = np.dot(word_vector_1, word_vector_2)
        theta_den = np.linalg.norm(word_vector_1) * np.linalg.norm(word_vector_2)
        theta = theta_sum / theta_den
        
        word = index_to_word[j]
        word_similarity[word] = theta
    
    return word_similarity #words_sorted

In [12]:
data = pd.read_csv('data/train_data_cat.csv')
data.postal_code = data.postal_code.astype(str)
data['postal_code'] = data['postal_code'].str.zfill(5)
data = data.drop(['accountid','userid'],axis=1)

In [13]:
data['data'] = data.apply(' '.join, axis=1)

In [14]:
texts = pd.DataFrame()
texts['data'] = data['data']

In [15]:
text_lines = []
for index, row in texts.iterrows():
    text_lines.append(row['data'].split(" "))

In [16]:
text_tokens = [item for text_lines in text_lines for item in text_lines]
texts = list(np.unique(np.array(text_tokens)))
VOCAB_SIZE = len(texts)

In [17]:
print(VOCAB_SIZE)

12171


In [18]:
# create the tokenizer
tokenizer = Tokenizer(filters=' ')
# fit the tokenizer on the documents
tokenizer.fit_on_texts(texts)

In [19]:
encoded_docs = tokenizer.texts_to_sequences(text_lines)

In [20]:
# integer encode the documents
vocab_size = len(tokenizer.word_index) + 1
num_classes = vocab_size

In [21]:
padded_docs = pad_sequences(encoded_docs, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

In [22]:
train = pd.DataFrame(padded_docs)
train_generator = DataGenerator(df_X=train, batch_size=batch_size, shuffle=True)
STEP_SIZE_TRAIN=train_generator.n()//train_generator.batch_size

In [23]:
model = create_classifier(learning_rate)
        
lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'loss', factor = 0.4, patience = 3, verbose = 0, min_delta = 0.001, mode = 'min')
es = tf.keras.callbacks.EarlyStopping(monitor='loss', mode='min', verbose=0, patience=15, restore_best_weights=True)
mc = tf.keras.callbacks.ModelCheckpoint(ckp_path, monitor='loss', mode='min', verbose=0, save_best_only=True, save_weights_only=True)
        
# train the model
history = model.fit(x=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    batch_size=batch_size,
                    epochs=num_epochs,
                    callbacks=[mc,lr,es],  
                    shuffle=True)


Epoch 1/500
153/153 [==============================] - 67s 336ms/step - loss: 5.3768 - lr: 0.0010
Epoch 2/500
153/153 [==============================] - 51s 333ms/step - loss: 5.4299 - lr: 0.0010
Epoch 3/500
153/153 [==============================] - 52s 339ms/step - loss: 4.7672 - lr: 0.0010
Epoch 4/500
153/153 [==============================] - 52s 339ms/step - loss: 4.5358 - lr: 0.0010
Epoch 5/500
153/153 [==============================] - 52s 339ms/step - loss: 4.3632 - lr: 0.0010
Epoch 6/500
153/153 [==============================] - 52s 340ms/step - loss: 4.1611 - lr: 0.0010
Epoch 7/500
153/153 [==============================] - 52s 340ms/step - loss: 3.9275 - lr: 0.0010
Epoch 8/500
153/153 [==============================] - 52s 340ms/step - loss: 3.7312 - lr: 0.0010
Epoch 9/500
153/153 [==============================] - 52s 340ms/step - loss: 3.5353 - lr: 0.0010
Epoch 10/500
153/153 [==============================] - 52s 341ms/step - loss: 3.3214 - lr: 0.0010
Epoch 11/500
153/15

Epoch 84/500
153/153 [==============================] - 52s 341ms/step - loss: 0.0255 - lr: 0.0010
Epoch 85/500
153/153 [==============================] - 52s 341ms/step - loss: 0.0222 - lr: 0.0010
Epoch 86/500
153/153 [==============================] - 51s 336ms/step - loss: 0.0238 - lr: 0.0010
Epoch 87/500
153/153 [==============================] - 51s 335ms/step - loss: 0.0255 - lr: 0.0010
Epoch 88/500
153/153 [==============================] - 52s 342ms/step - loss: 0.0197 - lr: 0.0010
Epoch 89/500
153/153 [==============================] - 53s 343ms/step - loss: 0.0159 - lr: 0.0010
Epoch 90/500
153/153 [==============================] - 51s 337ms/step - loss: 0.0163 - lr: 0.0010
Epoch 91/500
153/153 [==============================] - 51s 336ms/step - loss: 0.0197 - lr: 0.0010
Epoch 92/500
153/153 [==============================] - 52s 342ms/step - loss: 0.0143 - lr: 0.0010
Epoch 93/500
153/153 [==============================] - 51s 336ms/step - loss: 0.0183 - lr: 0.0010
Epoch 94/5

In [24]:
model = create_classifier(learning_rate)

In [25]:
model.load_weights(ckp_path)

In [26]:
embedding_layer = model.get_layer('embeddings')

In [27]:
embeddings = embedding_layer.get_weights()[0]

In [28]:
np.savetxt("data/embeddings_model_w2v_v3.csv", embeddings, delimiter=",")

In [29]:
word_to_index = tokenizer.word_index
index_to_word = dict()

for key in word_to_index:
    index_to_word.update({word_to_index[key] : key })

word_to_index.update({'unk':0})
index_to_word.update({0:'unk'})  

In [30]:
import pickle

with open('data/word_to_index_v3.pkl', 'wb') as fp:
    pickle.dump(word_to_index, fp)
    
with open('data/index_to_word_v3.pkl', 'wb') as fp:
    pickle.dump(index_to_word, fp)    

In [31]:
#embeddings[tokenizer.word_index['unk']]

In [32]:
cosine_similarity(embeddings[word_to_index['il']].reshape(1,300),embeddings[word_to_index['or']].reshape(1,300))[0][0]

0.13304226

In [33]:
cosine_similarity(embeddings[word_to_index['60126']].reshape(1,300),embeddings[word_to_index['60181']].reshape(1,300))[0][0]

0.013190172

In [34]:
cosine_similarity(embeddings[word_to_index['60126']].reshape(1,300),embeddings[word_to_index['97035']].reshape(1,300))[0][0]

-0.06738729

In [35]:
il_60126 = np.reshape(embeddings[word_to_index['il']],embeddings_shape) + np.reshape(embeddings[word_to_index['60126']],embeddings_shape)
il_60181 = np.reshape(embeddings[word_to_index['il']],embeddings_shape) + np.reshape(embeddings[word_to_index['60181']],embeddings_shape)
or_97035 = np.reshape(embeddings[word_to_index['or']],embeddings_shape) + np.reshape(embeddings[word_to_index['97035']],embeddings_shape)

In [36]:
cosine_similarity(il_60126,il_60181)[0][0]

0.6812738

In [37]:
cosine_similarity(il_60126,or_97035)[0][0]

0.038763817

In [38]:
il_60126 += np.reshape(embeddings[word_to_index['dupage'],:],embeddings_shape)
il_60181 += np.reshape(embeddings[word_to_index['dupage'],:],embeddings_shape)
or_97035 += np.reshape(embeddings[word_to_index['clackamas'],:],embeddings_shape)

In [39]:
cosine_similarity(il_60126,il_60181)[0][0]

0.81761783

In [40]:
cosine_similarity(il_60126,or_97035)[0][0]

0.26062876